# 03 - Convolutional Neural Networks And Computer Vision with TensorFlow

In the prievious notebook we've covered the basics of TensorFlow and built some models to work across different problems.

Now we're going to get specific and see how a special kind of neural network, **Convolutional neural networks (CNNs)** can be used for computer vision (detecting patterns in visual data).

> **Note:** In deep learning, many different kinds of model architectures can be used for different problems. For example, you could use a convolutional neural network for making predictions on image data and/or text data. However, in practice some architectures typically work better than others.

For example, you might want to:
- Classify whether a picture of fodd contains pizza 🍕 or steak 🥩
- Detect whether or not an object appears in an image.

## What we're going to cover

Specifically, we're going to go through the follow with TensorFlow:

- Getting a dataset to work with
- Architecture of a convolutional neural network
- A quick end-to-end example
- Steps in modelling for binary image classification with CNNs
    - Becoming one with the data
    - Preparing data for modelling
    - Creating a CNN model
    - Fitting a model
    - Evaluating a model
    - Improving a model
    - Making a prediction with a trained model
- Steps in modelling for multi-class image classification with CNNs
    - Same as above (but this time with a different dataset)

## A (typical) architecture of a convolutional neural network

Convolutional neural networks are no different to other kinds of deep learning neural networks in the fact they can be created in many different ways. Whate you see below are some components you'd expect to find in a traditional CNN.

Components of a convolutional neural network:

| **Hyperparameter/Layer type** | **What does it do?** | **Typical values** |
| ----- | ----- | ----- |
| Input images(s) | Target images you'd like to discover patterns in | Whatever you can take a photo (or video) of |
| Input layer | Takes in target images and preprocesses them for further layers | `input_shape = [batch_size,image_height, image_width, color_channels]` |
| Convolution layer | Extracts/learns the most important features from target images | Multiple, can create with `tf.Keras.layers.ConvXD` (X can be multiple values) |
| Hidden activation | Adds non-linearity to learned features (non-straight lines) | Usually ReLU (`tf.keras.activations.relu`) |
| Pooling layer | Reduces the dimensionality of learned image features | Average (`tf.keras.layers.AvgPool2D`) or Max (`tf.keras.layers.MaxPool2D`) |
| Fully connected layer | Further refines learned features from convolutional layers | `tf.keras.layers.Dense` |
| Output layer | Takes learned features and outputs them in shape of target labels | `output_shape = [number_of_classes]` (e.g. 3 for pizza, steak or sushi) |
| Output activation | Adds non-linearities to output layer | `tf.keras.activations.sigmoid` (binary classification) or `tf.keras.activations.softmax` |

Example stack together : 

<center><img src = "images/03-simple-convnet.png" width = 800px></center>

## Get the data

Because convolutional neural networks work so well with images, to learn more about them, we're going to start with a dataset of images.

The images we're going to work with are from the [Food-101 dataset](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/), a collection of 101 different categories of 101,000 (1000 images per category) real-world images of food dishes.

To begin, we're only going to use two of the categories, pizza 🍕 and steak 🥩 and build a binary classifier.

> 🔑 **Note:** To prepare the data we're using, preprocessing steps such as, moving the images into different subset folders, have been done. To see these preprocessing steps check out the preprocessing notebook.


We'll download the `pizza_steak` subset .zip file and unzip it.

In [4]:
# Install wget
# !pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=c6784904cfb2b23c20f87886696cae0017dbecfc6b8da4f9531f4880c0537041
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [1]:
import zipfile
import wget

# Download zip file of pizza_steak images
path = "data/dataset"
url = "https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip"
wget.download(url, out = path)

'data/dataset/pizza_steak.zip'

In [2]:
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("data/dataset/pizza_steak.zip", "r")
zip_ref.extractall("data/dataset")
zip_ref.close()

## Inspect the data (become one with it)

A very crucial step at the beginning of any machine learning project is becoming one with the data. This usually means plenty of visualizing and folder scanning to understand the data you're working with.

The file structure has been formatted to be in a typical format you might use for working with images.

More specifically:
- A **train** directory which contains all of the images in the training dataset with subdirectories each named after a certain class containing images of that class.

- A **test** directory with the same structure as the **train** directory.

Let's inspect each of the directories we've downloaded.

To do so, we can use the command `ls` which stands for list.

In [10]:
ls "data/dataset/pizza_steak"

 Le volume dans le lecteur C s'appelle Windows
 Le num�ro de s�rie du volume est A298-0804

 R�pertoire de c:\Users\user\Desktop\Machine Learning - Deep Learning\tensorflow-deep-learning\data\dataset\pizza_steak

26/05/2022  16:04    <DIR>          .
26/05/2022  16:04    <DIR>          ..
26/05/2022  16:04    <DIR>          test
26/05/2022  16:04    <DIR>          train
               0 fichier(s)                0 octets
               4 R�p(s)  518�317�281�280 octets libres


We can see we've got a train and test folder

In [11]:
ls "data/dataset/pizza_steak/train"

 Le volume dans le lecteur C s'appelle Windows
 Le num�ro de s�rie du volume est A298-0804

 R�pertoire de c:\Users\user\Desktop\Machine Learning - Deep Learning\tensorflow-deep-learning\data\dataset\pizza_steak\train

26/05/2022  16:04    <DIR>          .
26/05/2022  16:04    <DIR>          ..
26/05/2022  16:04    <DIR>          pizza
26/05/2022  16:04    <DIR>          steak
               0 fichier(s)                0 octets
               4 R�p(s)  518�317�113�344 octets libres


In [12]:
ls "data/dataset/pizza_steak/train/steak"

 Le volume dans le lecteur C s'appelle Windows
 Le num�ro de s�rie du volume est A298-0804

 R�pertoire de c:\Users\user\Desktop\Machine Learning - Deep Learning\tensorflow-deep-learning\data\dataset\pizza_steak\train\steak

26/05/2022  16:04    <DIR>          .
26/05/2022  16:04    <DIR>          ..
26/05/2022  16:04            36�185 1000205.jpg
26/05/2022  16:04            34�497 100135.jpg
26/05/2022  16:04           116�802 101312.jpg
26/05/2022  16:04            56�754 1021458.jpg
26/05/2022  16:04            34�143 1032846.jpg
26/05/2022  16:04            24�688 10380.jpg
26/05/2022  16:04            37�134 1049459.jpg
26/05/2022  16:04            49�841 1053665.jpg
26/05/2022  16:04            59�983 1068516.jpg
26/05/2022  16:04            28�996 1068975.jpg
26/05/2022  16:04            29�525 1081258.jpg
26/05/2022  16:04            49�577 1090122.jpg
26/05/2022  16:04            59�976 1093966.jpg
26/05/2022  16:04            60�253 1098844.jpg
26/05/2022  16:04            6

We have a whole bunch of images. It seems we have 750 images.